# Álgebra Linear Computacional - CKP8122 - MDCC - UFC
### Francisco Mateus dos Anjos Silva
# Método QR para Autovalores e Autovetores

As transformações de Householder são amplamente utilizadas na álgebra linear numérica, para realizar decomposições QR e são o primeiro passo do algoritmo QR. Elas também são amplamente utilizadas para a tridiagonalização de matrizes simétricas e para a transformação de matrizes não-simétricas para a forma de Hessenberg.

**Decomposição QR**

As transformações de Householder podem ser usadas para calcular a decomposição QR refletindo primeiramente uma coluna de uma matriz sobre um múltiplo de um vetor da base canônica, calculando a matriz de transformação, multiplicando-a com a matriz original e, então, continuando recursivamente com os menores ${\textstyle (i,i)}$ daquele produto.

**Referências:**

- https://pt.wikipedia.org/wiki/Transforma%C3%A7%C3%A3o_de_Householder#Decomposi%C3%A7%C3%A3o_QR
- https://sites.icmc.usp.br/andretta/ensino/aulas/sme0301-1-10/AutovaloresFrancis.pdf

In [1]:
# Implemente o método QR para achar os autovalores e autovetores de matrizes.

# I.  Matrizes simétricas
#     - Entre com uma matriz simétrica A nxn e uma tolerância
#     - Aplique o método QR diretamente sobre a matriz de entrada e encontre a matriz final (Diagonal) contendo os 
#       autovalores e a matriz acumulada Q = Q1 . Q2 . Q3 . Q4 ... cujas colunas são os autovetores da matriz 
#       original.
#     - Aplique o método de Householder primeiro (primeira caixa preta) e depois o método QR sobre a saída do 
#       Método de Householder, obtendo a matriz diagonal que contém os autovalores da matriz original. Obtenhas 
#       os autovetores da matriz original como colunas da matriz P = H Q, onde H é a matriz acumulada de Householder 
#       e Q é a matriz acumulada no método QR.

# II.  Matrizes não simétricas
#      - Repita o que foi feito para matrizes simétricas. Porém, a matriz final do método QR é uma matriz Triangular 
#        Superior e Blocos (BUT).
#      - Encontre os autovalores da matriz original, achando os autovalores dos blocos 2x2 ao longo da diagonal da BUT.
#      - Ache os autovetores da matriz BUT e, em seguida, os autovetores da matriz original.

In [2]:
import numpy as np
import math

In [3]:
def mount_householder_matrix(A, j):    
    A = A.astype(np.float32)
    n_A = len(A)
    v = np.zeros(n_A)
    w = np.zeros(n_A)
    v[j+1:] = A[j+1:,j]
    w[j+1] = np.linalg.norm(v)
    n = np.array([(v - w) / np.linalg.norm(v - w)])
    H = np.identity(n_A) - 2 * (n.T @ n)
    H = np.around(H, decimals=5)
    return H

In [4]:
def householder_method(A):
    #print('Matriz original:\n', A)
    n_A = len(A)
    H_acc = np.identity(n_A)
    matrix = A.copy()
    
    for j in range(0, n_A - 2):
        Hj = mount_householder_matrix(matrix, j)
        #print('\n\nMatriz de Householder ( passo', j, '):\n', Hj)

        matrix = np.around(Hj @ matrix @ Hj, decimals=4)
        #print('\nMatriz modificada pela transformação de similaridade ( passo', j, '):\n', matrix)
        
        H_acc = np.around(H_acc @ Hj, decimals=4)
        #print('\nMatriz de Householder acumulada ( passo', j, '):\n', H_acc)
    
    return matrix, H_acc

In [5]:
def mount_householder_matrix_to_qr(A, j):
    A = A.astype(np.float32)
    n_A = len(A)
    v = np.zeros(n_A)
    w = np.zeros(n_A)
    v[j:] = A[j:,j]
    w[j] = np.linalg.norm(v)
    
    if v[j] > 0:
        w[j] = -w[j]
    n = np.array([(v - w) / np.linalg.norm(v - w)])
    H = np.identity(n_A) - 2 * (n.T @ n)
    H = np.around(H, decimals=5)
    return H

In [6]:
def qr_decomposition(A):
    n_A = len(A)
    Q = np.identity(n_A)
    R = A.copy()
    
    for j in range(0, n_A - 1):
        Qj = mount_householder_matrix_to_qr(R, j)
        R = Qj @ R
        Q = Q @ Qj        
    return Q, R

In [7]:
def qr_method_eigvalues_eigvectors(A, eps=0.0001):
    n_A = len(A)
    P = np.identity(n_A)
    dif_between_ite = 0
    qtd_ite = 0
    
    while True:
        Q, R = qr_decomposition(A)
        A = R @ Q
        P = P @ Q
        
        d_A = np.diagonal(A)
        norm_d_A = np.linalg.norm(d_A)
        
        if math.fabs((norm_d_A - dif_between_ite)) <= eps:
            break;
        else:
            dif_between_ite = norm_d_A;
        
        qtd_ite+=1
        
        if qtd_ite > 400:
            break;
      
    return A, P

## 1. Teste com Matriz Simétrica 
### 1.1 Autovalores e autovetores usando diretamente o método QR

In [13]:
np.set_printoptions(precision=4, linewidth=300)

# Matriz 7x7
A = np.array([[ 3,-2, 1, 1,-2, 0, 2],
              [-2, 3, 0, 2, 1, 0, 0],
              [ 1, 0, 2, 0, 2, 1, 1],
              [ 1, 2, 0, 3, 0, 0, 2],
              [-2, 1, 2, 0, 3, 1, 0],
              [ 0, 0, 1, 0, 1, 1,-1],
              [ 2, 0, 1, 2, 0,-1, 1]])

D, P = qr_method_eigvalues_eigvectors(A)

print('Matriz A:\n', A)
print('\nMatriz diagonal D dos autovalores:\n', D)
print('\nAutovalores:\n', np.diagonal(D))
print('\nMatriz P dos autovetores:\n', P)

print('\n\nAutovalores (numpy):\n',np.linalg.eigvals(A))
print('\nAutovetores (numpy):\n',np.linalg.eig(A)[1])

print('\nVerificação dos autovalores e autovetores da matriz A:\n')
for i in range(0,len(A)):
    print('A * P[',i,']:', A @ P[:,i])
    print('lambda[',i,'] * P[',i,']:', D[i,i] * P[:,i])
    print('\n')

Matriz A:
 [[ 3 -2  1  1 -2  0  2]
 [-2  3  0  2  1  0  0]
 [ 1  0  2  0  2  1  1]
 [ 1  2  0  3  0  0  2]
 [-2  1  2  0  3  1  0]
 [ 0  0  1  0  1  1 -1]
 [ 2  0  1  2  0 -1  1]]

Matriz diagonal D dos autovalores:
 [[ 6.8117e+00 -2.4058e-05 -1.2620e-05  3.6018e-05 -1.2394e-05  7.8919e-06  8.3890e-06]
 [-2.4058e-05  5.7796e+00  2.4436e-05 -5.8955e-06 -1.4461e-05 -1.2251e-05  2.4108e-05]
 [-1.2620e-05  2.4436e-05  4.4530e+00  1.1166e-05 -1.2241e-05 -1.7440e-07  7.9911e-06]
 [ 3.6018e-05 -5.8955e-06  1.1166e-05 -1.3964e+00 -6.3049e-06  4.3939e-06  4.3558e-06]
 [-1.2394e-05 -1.4461e-05 -1.2241e-05 -6.3049e-06  1.1325e+00 -6.8122e-02  3.2730e-08]
 [ 7.8919e-06 -1.2251e-05 -1.7440e-07  4.3939e-06 -6.8122e-02 -1.0338e+00 -2.5359e-06]
 [ 8.3890e-06  2.4108e-05  7.9911e-06  4.3558e-06  3.2730e-08 -2.5359e-06  2.5376e-01]]

Autovalores:
 [ 6.8117  5.7796  4.453  -1.3964  1.1325 -1.0338  0.2538]

Matriz P dos autovetores:
 [[-0.6536 -0.2096 -0.2395 -0.3057  0.1673 -0.5729 -0.1478]
 [ 0.4376 -0.

### 1.2 Autovalores e autovetores usando a transformação de Householder antes do método QR

In [9]:
np.set_printoptions(precision=4, linewidth=300)

# Matriz 7x7
A = np.array([[ 3,-2, 1, 1,-2, 0, 2],
              [-2, 3, 0, 2, 1, 0, 0],
              [ 1, 0, 2, 0, 2, 1, 1],
              [ 1, 2, 0, 3, 0, 0, 2],
              [-2, 1, 2, 0, 3, 1, 0],
              [ 0, 0, 1, 0, 1, 1,-1],
              [ 2, 0, 1, 2, 0,-1, 1]])
print('Matriz A:\n', A)

matrix, H_acc = householder_method(A)
print('\nMatriz A após transformação de Householder (tridiagonal):\n',matrix)

D2, P2 = qr_method_eigvalues_eigvectors(matrix)
eigvectors_matrix = H_acc @ P2
print('\nMatriz diagonal dos autovalores:\n', D2)
print('\nAutovalores:\n', np.diagonal(D2))
print('\nMatriz dos autovetores: \n ', eigvectors_matrix)

print('\n\nVerificação dos autovalores e autovetores da matriz A:\n')

for i in range(0, len(A)):
    print('\nA * eigvectors_matrix[',i,']:', A @ eigvectors_matrix[:,i])
    print('lambda[',i,'] * eigvectors_matrix[',i,']:', D2[i,i] * eigvectors_matrix[:,i])

print('\nAutovalores (numpy):', np.linalg.eigvals(A))
print('\nAutovetores (numpy):\n', np.linalg.eig(A)[1])

Matriz A:
 [[ 3 -2  1  1 -2  0  2]
 [-2  3  0  2  1  0  0]
 [ 1  0  2  0  2  1  1]
 [ 1  2  0  3  0  0  2]
 [-2  1  2  0  3  1  0]
 [ 0  0  1  0  1  1 -1]
 [ 2  0  1  2  0 -1  1]]

Matriz A após transformação de Householder (tridiagonal):
 [[ 3.      3.7416  0.      0.      0.      0.      0.    ]
 [ 3.7416  2.6428  1.1089  0.      0.      0.      0.    ]
 [ 0.      1.1089  3.1123  2.1448  0.      0.      0.    ]
 [ 0.      0.      2.1448  2.2208  1.5165  0.      0.    ]
 [ 0.      0.      0.      1.5165  1.9329  2.8556 -0.    ]
 [ 0.      0.      0.      0.      2.8556  2.7001  0.9078]
 [ 0.      0.      0.      0.     -0.      0.9078  0.391 ]]

Matriz diagonal dos autovalores:
 [[ 6.8115e+00  3.3305e-04  2.0028e-06 -1.9113e-06  2.6663e-06  6.8482e-06 -6.9225e-07]
 [ 3.3305e-04  5.7794e+00 -6.9930e-05  3.4435e-06  1.4642e-05 -4.6124e-06  3.3665e-06]
 [ 2.0028e-06 -6.9930e-05  4.4530e+00  1.5249e-06 -5.0204e-06 -1.5190e-06  1.1015e-06]
 [-1.9113e-06  3.4435e-06  1.5249e-06 -1.3961e+00 

## 2. Teste com Matriz Não Simétrica 
### 2.1 Autovalores e autovetores usando a transformação de Householder antes do método QR

In [10]:
np.set_printoptions(precision=4, linewidth=300)

# Matriz 7x7
A = np.array([[3,-2, 1,-1, 2, 0, 2],
              [1, 3, 0, 2, 1, 0, 0],
              [5, 0, 2, 0, 2, 1, 1],
              [1, 5, 0, 3, 0, 0, 2],
              [2, 1, 3, 5, 3, 1, 0],
              [3, 5, 1, 0, 2, 1, 1],
              [2, 0, 5, 2,-3, 0, 1]])
print('Matriz A:\n', A)

matrix, H_acc = householder_method(A)
print('\nMatriz A após transformação de Householder (Hessenberg):\n',H_acc)

Q3, R3 = qr_method_eigvalues_eigvectors(matrix)
print('\nMatriz Q gerada pela decomposição QR da matriz de Hessenberg obtida:\n', Q3)
print('\nAutovalores (numpy):\n',np.linalg.eigvals(A))
print('\nAutovetores (numpy):\n',np.linalg.eig(A)[1])

print('\nCálculo dos autovalores a partir dos blocos da matriz de Hessenberg:\n')
diag_index = 0
while diag_index < (len(Q3)-1):
    # Se o valor abaixo é aproximadamente zero, então o elemento é autovalor
    if np.isclose(Q3[diag_index+1,diag_index], 0.0, rtol=0, atol=1e-04):
        print('\nHessenberg[',diag_index,',',diag_index,'] é autovalor:', Q3[diag_index,diag_index])
        diag_index+=1
    # Caso contrário, os autovalores são as raízes do polinômio característico da matriz bloco 2x2
    else:
        print('\nMatrizes para cálculo de autovalores:\n', Q3[diag_index:diag_index+2,diag_index:diag_index+2])
        coeff = [1, 
                 -(Q3[diag_index,diag_index] + Q3[diag_index+1,diag_index+1]), 
                 (Q3[diag_index,diag_index] * Q3[diag_index+1,diag_index+1] - Q3[diag_index+1,diag_index] * Q3[diag_index,diag_index+1])]
        print('\nRaízes da equação (autovalores):\n', np.roots(coeff))
        diag_index+=2
        
# Pega o último elemento da diagonal caso ele tenha sobrado
if diag_index == (len(Q3)-1):
    print('\nHessenberg[',diag_index,',',diag_index,'] é autovalor:', Q3[diag_index,diag_index])

Matriz A:
 [[ 3 -2  1 -1  2  0  2]
 [ 1  3  0  2  1  0  0]
 [ 5  0  2  0  2  1  1]
 [ 1  5  0  3  0  0  2]
 [ 2  1  3  5  3  1  0]
 [ 3  5  1  0  2  1  1]
 [ 2  0  5  2 -3  0  1]]

Matriz A após transformação de Householder (Hessenberg):
 [[ 1.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.1508  0.0287  0.2266  0.5509  0.0987 -0.7823]
 [ 0.      0.7538 -0.5093 -0.0396 -0.0176 -0.4098  0.051 ]
 [ 0.      0.1508  0.2329  0.3283 -0.7902 -0.0637 -0.432 ]
 [ 0.      0.3015  0.7107  0.3464  0.2598 -0.3327  0.3255]
 [ 0.      0.4523  0.0046  0.1996 -0.0245  0.8368  0.2334]
 [ 0.      0.3015  0.4249 -0.8243 -0.0593  0.0843 -0.1962]]

Matriz Q gerada pela decomposição QR da matriz de Hessenberg obtida:
 [[ 8.9457e+00  2.6880e+00 -2.7957e+00 -2.5814e+00  2.5945e+00 -1.9253e+00  2.8625e+00]
 [-3.3735e-05  5.5119e+00  2.5506e+00 -1.2269e+00  9.9832e-01  6.5620e-01 -1.5478e+00]
 [-2.0143e-05  2.1983e-05  3.6063e+00  5.9101e-02 -1.4011e+00 -2.7534e+00 -2.6061e+00]
 [ 1.8844e-07 -